<a href="https://colab.research.google.com/github/Strojove-uceni/2024-final-letadylka-prochazka-belohlavek/blob/main/architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Architectures
This file contains the description of the main architectures used within this project. Below we provide a detailed description of them.

First we describe models that were used for Q-values predictions:

    - DGN
    - DQN
    - Comm_net,
    
then we go over the methods that were used to aggregate hidden graph representations:
    - SUM
    - GCN
and lastly we describe the NetMon class that was originally provided by the authors.




In [ ]:
ul

## Q-values prediction
something about..


### DGN
what is it, how it works, provide that code

### DQN

same

### Comm_net
same

## State aggregation

....

### SUM


### GCN
...

## NetMon